In [1]:
import os
import sys
# sys.path.append("../")
from datatools.analyzer import *

from datatools.maneger import DataManager
import pandas as pd

/home/yamada/.local/share/virtualenvs/twitchAI-AEfI_JA6/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
out_path = "../corpus/novel_formated/"
# corpus_name = "novel_segments.tsv"
corpus_name = "ntt_segment_plain2.tsv"

In [3]:
import csv
conv_data = []
with open(out_path+corpus_name, "r") as f:
    reader = csv.reader(f, delimiter="\t")
    conv_data = [row for row in reader]

# conv_data_mini = conv_data[:len(conv_data)//100]


In [4]:
# conv_data_mini = conv_data[:500]
lim=20000
import random
random.seed(0)
conv_data_mini = random.sample(conv_data, lim)

In [5]:
def make_Src_Tgt(conv_data):
    src_str = []
    tgt_str = []

    for conv in conv_data:
        src_str.append(conv[0])
        tgt_str.append(conv[1])
    return src_str, tgt_str

src_str, tgt_str = make_Src_Tgt(conv_data_mini)

In [6]:
for (s, t) in zip(src_str[:10], tgt_str[:10]):
    print(s, "\t", t)

ぜひぜひ、食べたい物ややりたいことを考えておいてください! 	 嬉しい!
以前、高知の大学に通ってた頃は犬を飼ってましたよ 	 一緒に暮らしている姉は犬好きでね、昔は家でも飼ってたわ。
占い師さんを目指してたんですね!わたしは占い師の助言をもとにして、いまパティシエをやってるんです! 	 ほうほう、洋菓子の職人さんですね。
私は程よい癖毛でパーマをあててるように見えるのが自分でも気に入ってます 	 素敵な髪ですね!
北海道は、夏でもそんなに暑くないし、御勧めです 	 ぜひぜひ!
坂本龍馬が好きなのね!外国人にもすっごく人だし 	 うれしい!
でもハムスターは場所もとらないし、鳴かないし、飼いやすいですよ 	 ペットショップに行ってみようかな、ハムスター飼ってみたくなりました。
そうそう、あなたはどんなお仕事なんです? 	 コンピューターのシステム関係の仕事をしています。
私、背が高いのがコンプレックスで、それもあるかな 	 誰でもコンプレックスはあるよー。
マラソンは部活でなければやってません 	 そうですね。


In [7]:
from collections import Counter
from torchtext.vocab import Vocab

def build_vocab(texts, tokenizer):
    
    counter = Counter()
    for text in tqdm(texts):
        counter.update(tokenizer(text))
    return Vocab(counter, specials=['<unk>', '<pad>','<fos>', '<eos>', '<del>'])

In [8]:
src_trainval_str, src_test_str, tgt_trainval_str, tgt_test_str= train_test_split(src_str, tgt_str, test_size=0.20, random_state=5)
print("len=> src_train_val:{0}, src_test:{1}".format(len(src_trainval_str), len(src_test_str)))

src_train_str, src_val_str, tgt_train_str, tgt_val_str= train_test_split(src_trainval_str, tgt_trainval_str, test_size=0.10, random_state=5)

print("len=> src_train:{0}, src_val:{1}".format(len(src_train_str), len(src_val_str)))

len=> src_train_val:16000, src_test:4000
len=> src_train:14400, src_val:1600


In [9]:
src_train_str_ = src_train_str[:lim]
tgt_train_str_ = tgt_train_str[:lim]
src_val_str_ = src_val_str[:lim//10]
tgt_val_str_ = tgt_val_str[:lim//10]

In [10]:
tokenizer_src = mecab_tokenize
tokenizer_tgt = mecab_tokenize

In [11]:
vocab_src = build_vocab(src_train_str_, tokenizer=tokenizer_src)
vocab_tgt = build_vocab(tgt_train_str_, tokenizer=tokenizer_tgt)

100%|██████████| 14400/14400 [00:00<00:00, 25692.31it/s]


In [12]:
PAD_IDX = vocab_src.stoi["<pad>"]
PAD_IDX

1

In [13]:
vocab_path = "../models/vocab/"
vocab_name = "vocab_transformer_src_ntt_lim={0}.pickle".format(lim)
dictM = DataManager(vocab_path)
dictM.save_data(vocab_name, vocab_src)
vocab_name = "vocab_transformer_tgt_ntt_lim={0}.pickle".format(lim)
dictM.save_data(vocab_name, vocab_tgt)

success save : ../models/vocab/vocab_transformer_src_ntt_lim=20000.pickle
success save : ../models/vocab/vocab_transformer_src_ntt_lim=20000.pickle
success save : ../models/vocab/vocab_transformer_tgt_ntt_lim=20000.pickle
success save : ../models/vocab/vocab_transformer_tgt_ntt_lim=20000.pickle


In [14]:
import torch
import torch.nn as nn
from torch import Tensor
from torch.nn import (
    TransformerEncoder, TransformerDecoder,
    TransformerEncoderLayer, TransformerDecoderLayer
)
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim
import torch.nn.utils.rnn as rnn

In [15]:
def data_process(texts_src, texts_tgt, vocab_src, vocab_tgt, tokenizer_src, tokenizer_tgt):
    
    data = []
    for (src, tgt) in zip(texts_src, texts_tgt):
        src_tensor = torch.tensor(
            convert_text_to_indexes(text=src, vocab=vocab_src, tokenizer=tokenizer_src),
            dtype=torch.long
        )
        tgt_tensor = torch.tensor(
            convert_text_to_indexes(text=tgt, vocab=vocab_tgt, tokenizer=tokenizer_tgt),
            dtype=torch.long
        )
        data.append((src_tensor, tgt_tensor))
        
    return data

def convert_text_to_indexes(text, vocab, tokenizer):
    return [vocab['<fos>']] + [
        vocab[token] for token in tokenizer(text.strip("\n"))
    ] + [vocab['<eos>']]

In [16]:
train_data = data_process(
    texts_src=src_train_str_, texts_tgt=tgt_train_str_,
    vocab_src=vocab_src, vocab_tgt=vocab_tgt,
    tokenizer_src=tokenizer_src, tokenizer_tgt=tokenizer_tgt
)
valid_data = data_process(
    texts_src=src_val_str_, texts_tgt=tgt_val_str_,
    vocab_src=vocab_src, vocab_tgt=vocab_tgt,
    tokenizer_src=tokenizer_src, tokenizer_tgt=tokenizer_tgt
)

In [17]:
print('インデックス化された文章')
print('Input: {}\nOutput: {}'.format(train_data[0][0], train_data[0][1]))
print('')

print('インデックス化された文章をもとに戻す')
print('Input: {}\nOutput: {}'.format(
    ' '.join([vocab_src.itos[x] for x in train_data[0][0]]),
    ' '.join([vocab_tgt.itos[x] for x in train_data[0][1]])
))

インデックス化された文章
Input: tensor([  2,  37,   8,  16,  15,   5, 552, 245,  10,  51,  12,  31, 273,  17,
        284,  19,   8,   3])
Output: tensor([   2,   46,    9,  446,   25, 1091,   26,   22,   11,   40,   13,    6,
          15,    7,  130, 2979,   17,  278,   17, 1226,  240,   84,   19,    5,
           3])

インデックス化された文章をもとに戻す
Input: <fos> いい です よ ね 、 狭い 場所 が 苦手 な ので 田舎 で よかっ た です <eos>
Output: <fos> 今 は 都会 で 商社マン を し て いる ん です が 、 やっぱり 定期的 に 田舎 に 帰り たく なり ます 。 <eos>


In [18]:
batch_size = 128
batch_size = 32
PAD_IDX = vocab_src['<pad>']
START_IDX = vocab_src['<fos>']
END_IDX = vocab_src['<eos>']

In [19]:
def generate_batch(data_batch):
    
    batch_src, batch_tgt = [], []
    for src, tgt in data_batch:
        batch_src.append(src)
        batch_tgt.append(tgt)
        
    # batch_src = pad_sequence(batch_src, padding_value=PAD_IDX, batch_first=True)
    # batch_tgt = pad_sequence(batch_tgt, padding_value=PAD_IDX, batch_first=True)
    batch_src = pad_sequence(batch_src, padding_value=PAD_IDX, batch_first=False)
    batch_tgt = pad_sequence(batch_tgt, padding_value=PAD_IDX, batch_first=False)
    
    return batch_src, batch_tgt

In [20]:
train_iter = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(valid_data, batch_size=batch_size, shuffle=True, collate_fn=generate_batch)

In [21]:
"""
バッチの数:(4) -> src/tgt:(2) -> バッチの大きさ:(128) -> 系列長  : batch_frst=True

"""

len(list(train_iter)[0][0])

41

In [22]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cuda:0' if not torch.cuda.is_available() else 'cpu')

model_dir_path = Path('model')
if not model_dir_path.exists():
    model_dir_path.mkdir(parents=True)

In [23]:
device

device(type='cuda', index=0)

In [24]:
import math
class TokenEmbedding(nn.Module):
    
    def __init__(self, vocab_size, embedding_size):
        
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size, padding_idx=PAD_IDX)
        self.embedding_size = embedding_size
        
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.embedding_size)

In [25]:
class PositionalEncoding(nn.Module):
    
    def __init__(self, embedding_size: int, dropout: float, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        
        den = torch.exp(-torch.arange(0, embedding_size, 2) * math.log(10000) / embedding_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        embedding_pos = torch.zeros((maxlen, embedding_size))
        embedding_pos[:, 0::2] = torch.sin(pos * den)
        embedding_pos[:, 1::2] = torch.cos(pos * den)
        embedding_pos = embedding_pos.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('embedding_pos', embedding_pos)

    def forward(self, x: Tensor):
        return self.dropout(x + self.embedding_pos[: x.size(0)])

In [26]:
def create_mask(src, tgt, PAD_IDX):
    
    seq_len_src = src.shape[0]
    seq_len_tgt = tgt.shape[0]

    mask_tgt = generate_square_subsequent_mask(seq_len_tgt, PAD_IDX)
    mask_src = torch.zeros((seq_len_src, seq_len_src), device=device).type(torch.bool)

    padding_mask_src = (src == PAD_IDX).transpose(0, 1)
    padding_mask_tgt = (tgt == PAD_IDX).transpose(0, 1)
    
    return mask_src, mask_tgt, padding_mask_src, padding_mask_tgt


def generate_square_subsequent_mask(seq_len, PAD_IDX):
    mask = (torch.triu(torch.ones((seq_len, seq_len), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == PAD_IDX, float(0.0))
    return mask




    

In [27]:
class Seq2SeqTransformer(nn.Module):
    
    def __init__(
        self, num_encoder_layers: int, num_decoder_layers: int,
        embedding_size: int, vocab_size_src: int, vocab_size_tgt: int,
        dim_feedforward:int = 512, dropout:float = 0.1, nhead:int = 8
        ):
        
        super(Seq2SeqTransformer, self).__init__()

        self.token_embedding_src = TokenEmbedding(vocab_size_src, embedding_size)
        self.positional_encoding = PositionalEncoding(embedding_size, dropout=dropout)
        encoder_layer = TransformerEncoderLayer(
            d_model=embedding_size, nhead=nhead, dim_feedforward=dim_feedforward
        )
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        
        self.token_embedding_tgt = TokenEmbedding(vocab_size_tgt, embedding_size)
        decoder_layer = TransformerDecoderLayer(
            d_model=embedding_size, nhead=nhead, dim_feedforward=dim_feedforward
        )
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)
        
        self.output = nn.Linear(embedding_size, vocab_size_tgt)

    def forward(
        self, src: Tensor, tgt: Tensor,
        mask_src: Tensor, mask_tgt: Tensor,
        padding_mask_src: Tensor, padding_mask_tgt: Tensor,
        memory_key_padding_mask: Tensor
        ):
        
        embedding_src = self.positional_encoding(self.token_embedding_src(src))
        memory = self.transformer_encoder(embedding_src, mask_src, padding_mask_src)
        embedding_tgt = self.positional_encoding(self.token_embedding_tgt(tgt))
        outs = self.transformer_decoder(
            embedding_tgt, memory, mask_tgt, None,
            padding_mask_tgt, memory_key_padding_mask
        )
        return self.output(outs)

    def encode(self, src: Tensor, mask_src: Tensor):
        return self.transformer_encoder(self.positional_encoding(self.token_embedding_src(src)), mask_src)

    def decode(self, tgt: Tensor, memory: Tensor, mask_tgt: Tensor):
        return self.transformer_decoder(self.positional_encoding(self.token_embedding_tgt(tgt)), memory, mask_tgt)

In [28]:
def train(model, data, optimizer, criterion, PAD_IDX):
    
    model.train()
    losses = 0
    for src, tgt in tqdm(data):
        
        src = src.to(device)
        tgt = tgt.to(device)

        input_tgt = tgt[:-1, :]

        mask_src, mask_tgt, padding_mask_src, padding_mask_tgt = create_mask(src, input_tgt, PAD_IDX)

        # print(src.shape, tgt.shape)

        logits = model(
            src=src, tgt=input_tgt,
            mask_src=mask_src, mask_tgt=mask_tgt,
            padding_mask_src=padding_mask_src, padding_mask_tgt=padding_mask_tgt,
            memory_key_padding_mask=padding_mask_src
        )

        optimizer.zero_grad()

        output_tgt = tgt[1:, :]
        loss = criterion(logits.reshape(-1, logits.shape[-1]), output_tgt.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

        del logits
        del loss
        
    return losses / len(data)


def evaluate(model, data, criterion, PAD_IDX):
    
    model.eval()
    losses = 0
    for src, tgt in data:
        
        src = src.to(device)
        tgt = tgt.to(device)

        input_tgt = tgt[:-1, :]

        mask_src, mask_tgt, padding_mask_src, padding_mask_tgt = create_mask(src, input_tgt, PAD_IDX)

        logits = model(
            src=src, tgt=input_tgt,
            mask_src=mask_src, mask_tgt=mask_tgt,
            padding_mask_src=padding_mask_src, padding_mask_tgt=padding_mask_tgt,
            memory_key_padding_mask=padding_mask_src
        )
        
        output_tgt = tgt[1:, :]
        loss = criterion(logits.reshape(-1, logits.shape[-1]), output_tgt.reshape(-1))
        losses += loss.item()
        
    return losses / len(data)

In [29]:
vocab_size_src = len(vocab_src)
vocab_size_tgt = len(vocab_tgt)
embedding_size = 240
nhead = 8
dim_feedforward = 100
num_encoder_layers = 2
num_decoder_layers = 2
dropout = 0.1

model = Seq2SeqTransformer(
    num_encoder_layers=num_encoder_layers,
    num_decoder_layers=num_decoder_layers,
    embedding_size=embedding_size,
    vocab_size_src=vocab_size_src, vocab_size_tgt=vocab_size_tgt,
    dim_feedforward=dim_feedforward,
    dropout=dropout, nhead=nhead
)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

model = model.to(device)

criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(model.parameters())

In [30]:
import time

epoch = 30
best_loss = float('Inf')
best_model = None
patience = 10
counter = 0

for loop in range(1, epoch + 1):
    
    start_time = time.time()
    
    loss_train = train(
        model=model, data=train_iter, optimizer=optimizer,
        criterion=criterion, PAD_IDX=PAD_IDX
    )
    
    elapsed_time = time.time() - start_time
    
    loss_valid = evaluate(
        model=model, data=valid_iter, criterion=criterion, PAD_IDX=PAD_IDX
    )
    
    print('[{}/{}] train loss: {:.2f}, valid loss: {:.2f}  [{}{:.0f}s] count: {}, {}'.format(
        loop, epoch,
        loss_train, loss_valid,
        str(int(math.floor(elapsed_time / 60))) + 'm' if math.floor(elapsed_time / 60) > 0 else '',
        elapsed_time % 60,
        counter,
        '**' if best_loss > loss_valid else ''
    ))
    
    if best_loss > loss_valid:
        best_loss = loss_valid
        best_model = model
        counter = 0
        
    if counter > patience:
        break
    
    counter += 1

100%|██████████| 450/450 [01:02<00:00,  7.15it/s]


[1/30] train loss: 4.09, valid loss: 3.57  3s] count: 0, **


100%|██████████| 450/450 [01:04<00:00,  7.03it/s]


[2/30] train loss: 3.31, valid loss: 3.44  4s] count: 1, **


100%|██████████| 450/450 [01:03<00:00,  7.13it/s]


...

[8/30] train loss: 2.03, valid loss: 3.91  4s] count: 6, 


100%|██████████| 450/450 [01:03<00:00,  7.14it/s]


[9/30] train loss: 1.91, valid loss: 4.02  3s] count: 7, 


 37%|███▋      | 166/450 [00:23<00:39,  7.15it/s]


KeyboardInterrupt: 

In [31]:
torch.save(best_model.state_dict(), model_dir_path.joinpath('translation_transfomer_ntt.pth'))

In [32]:
model_path = "../models/transformer/"
model_name = "transformer_ntt_lim={0}.pickle".format(lim)
modelM = DataManager(model_path)
modelM.save_data(model_name, best_model)

success save : ../models/transformer/transformer_ntt_lim=20000.pickle
success save : ../models/transformer/transformer_ntt_lim=20000.pickle


In [33]:
def clean_text_plain(text):
    text_ = neologdn.normalize(text)
    # text_ = re.sub(r'\([^\)]*\)', "", text_)
    # text_ = re.sub(r'\([^\)]*\)', "", text_)
    text_ = re.sub(r'\d+', "0", text_)
    if "……" in text_:
        text_ = text_.replace("……", "…")
    return text_

In [34]:
def translate(
    model, text, vocab_src, vocab_tgt, tokenizer_src, seq_len_tgt, PAD_IDX, 
    START_IDX, END_IDX
):
    
    model.eval()
    tokens = convert_text_to_indexes(text=text, vocab=vocab_src, tokenizer=tokenizer_src)
    num_tokens = len(tokens)
    src = torch.LongTensor(tokens).reshape(num_tokens, 1)
    mask_src = (torch.zeros(num_tokens, num_tokens)).type(torch.bool) 
    # print(mask_src)
    predicts = beam_search_decode(
        model=model, src=src,
        mask_src=mask_src, seq_len_tgt=seq_len_tgt, PAD_IDX=PAD_IDX,
        START_IDX=START_IDX, END_IDX=END_IDX
    )
    
    for ys in predicts:
        # print(ys[2].prev_node)
        line = ' '.join([vocab_tgt.itos[token] for token in ys[2].prev_node.flatten()]).replace("<fos>", "").replace("<eos>", "")
        print(line)
    # return ' '.join([vocab_tgt.itos[token] for token in predicts]).replace("<fos>", "").replace("<eos>", "")


def greedy_decode(model, src, mask_src, seq_len_tgt, PAD_IDX, START_IDX, END_IDX):
    
    src = src.to(device)
    mask_src = mask_src.to(device)

    memory = model.encode(src, mask_src)
    # print(memory.shape)
    memory = model.transformer_encoder(model.positional_encoding(model.token_embedding_src(src)), mask_src)
    ys = torch.ones(1, 1).fill_(START_IDX).type(torch.long).to(device)
    # print(ys)
    
    for i in range(seq_len_tgt - 1):
        memory = memory.to(device)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(device).type(torch.bool)
        mask_tgt = (generate_square_subsequent_mask(ys.size(0), PAD_IDX).type(torch.bool)).to(device)

        output = model.decode(ys, memory, mask_tgt)
        # print(output)
        output = output.transpose(0, 1)
        output = model.output(output[:, -1])
        _, next_word = torch.max(output, dim = 1)
        next_word = next_word.item()
        # print(next_word)

        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == END_IDX:
            break
            
    return ys

In [35]:
from heapq import heappush, heappop
import copy

class BeamSearchNode(object):
    def __init__(self, h, prev_node, wid, logp, length):
        self.h = h
        self.prev_node = prev_node
        self.wid = wid
        self.logp = logp
        self.length = length

    def eval(self):
        return self.logp / float(self.length - 1 + 1e-6)

def beam_search_decode(model, src, mask_src, seq_len_tgt, PAD_IDX, START_IDX, END_IDX, width=10, n_best=15):
    n_best_list = []

    src = src.to(device)
    mask_src = mask_src.to(device)

    memory = model.encode(src, mask_src)
    # print(memory.shape)
    memory = model.transformer_encoder(model.positional_encoding(model.token_embedding_src(src)), mask_src)
    ys = torch.ones(1, 1).fill_(START_IDX).type(torch.long).to(device)

    # 開始
    node = BeamSearchNode(h=None, prev_node=copy.deepcopy(ys), wid=copy.deepcopy(ys), logp=0, length=1)
    # ビームサーチのノード全体
    nodes = []
    # EOSまでたどり着いたノード全体
    end_nodes = []

    heappush(nodes, (-node.eval(), id(node), node))
    n_dec_steps = 0
    while True:
        # 発話長
        if n_dec_steps > 20:
            break
        
        score, _, n = heappop(nodes)

        # <end>ならば終了
        if n.wid.item() == END_IDX and n.prev_node is not None:
            end_nodes.append((score, id(n), n))
            if len(end_nodes) >= n_best:
                    break
            else:
                continue
        
        mask_tgt = (generate_square_subsequent_mask(n.prev_node.size(0), PAD_IDX).type(torch.bool)).to(device)
        output = model.decode(n.prev_node, memory, mask_tgt)
        # print(output.shape)
        output = output.transpose(0, 1)
        # print(output.shape)
        output = model.output(output[:, -1])
        # print(output.shape)
        output = torch.log_softmax(output, dim=1)
        topk_log_prob, topk_indexes = torch.topk(output, width)
        # print(torch.max(output, dim = 1))
        # print(topk_log_prob ,topk_indexes)
        for new_k in range(width):
            next_word = topk_indexes[0][new_k]
            logp = topk_log_prob[0][new_k].item()
            ys_ = torch.cat([copy.deepcopy(n.prev_node), torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
            # print(ys_)
            node = BeamSearchNode(h=None, prev_node=copy.deepcopy(ys_), wid=next_word, logp=n.logp+logp, length=n.length+1)
            heappush(nodes, (-node.eval(), id(node), node))

        n_dec_steps += 1
        # break
    if len(end_nodes) == 0:
        print("can't reach end stage")
        end_nodes = [heappop(nodes) for _ in range(width)]
    
    # print(end_nodes[2][2].prev_node)
    return end_nodes

In [40]:
seq_len_tgt = max([len(x[1]) for x in train_data])

text = clean_text_plain('まあ、今は携帯でもすぐに案内してくれるし、安心ですよね')
print(text)

translate(
    model=model, text=text, vocab_src=vocab_src, vocab_tgt=vocab_tgt,
    tokenizer_src=tokenizer_src, seq_len_tgt=seq_len_tgt, PAD_IDX=PAD_IDX,
    START_IDX=START_IDX, END_IDX=END_IDX
)

まあ、今は携帯でもすぐに案内してくれるし、安心ですよね
 そう な ん です ね 。 
 そう な ん です ね ! 
 そう な ん です よ ね 。 
 そう な ん です よ 。 
 そう な ん です か 。 


In [37]:
src_test_str[:10]

['気楽な仕事なので卒業してからずっと同じ職場で働いています',
 'そういうのは個性だと思うな',
 '動物は飼ってません',
 '私雪が好きなんですが、京都の雪景色も綺麗そうですよね',
 '調べだしたら止まらないのが困るけど',
 'まあ、今は携帯でもすぐに案内してくれるし、安心ですよね',
 'わたしは鉄道マンなんですが殆ど体を動かしてないから見習わないとな',
 'たしかに言われてみるとそうですね、わたしは仕切りたがりだから今の仕事もすぐやめそう、あなたはどう?',
 'いや全然食べちゃいます!不規則な生活さえ正せば痩せる自信がありますし!',
 'パーマは気づいて欲しいね!わたしは和歌山出身だよ']